<a href="https://colab.research.google.com/github/vadigr123/colab_telegrams/blob/main/RVC_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone & Download RVC

In [ ]:
# @title Clone
!git clone https://github.com/IAHispano/Applio --branch 3.2.9 --single-branch
%cd /content/Applio

Cloning into 'Applio'...
remote: Enumerating objects: 17882, done.
remote: Counting objects: 100% (1077/1077), done.
remote: Compressing objects: 100% (345/345), done.
remote: Total 17882 (delta 934), reused 736 (delta 732), pack-reused 16805 (from 3)
Receiving objects: 100% (17882/17882), 30.69 MiB | 24.72 MiB/s, done.
Resolving deltas: 100% (11587/11587), done.
Note: switching to '2614ccdf56ed6672755576f767988f7a20a956a5'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

/content/Applio


In [ ]:
# @title Install

!sudo apt update
!sudo apt install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3
import sys
sys.path.append('/usr/local/lib/python3.10/dist-packages')

rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)

!pip install uv
from IPython.display import clear_output
clear_output()
print("Installing requirements...")
!uv pip install torch==2.7.0 torchvision torchaudio==2.7.0 --upgrade --index-url https://download.pytorch.org/whl/cu128 -q
!uv pip install -r requirements.txt -q
clear_output()
print("Finished installing requirements! ")

Finished installing requirements! 


In [ ]:
# @title Download models
!python core.py "prerequisites" --models "True" --exe "True"  --pretraineds_hifigan "True"

/usr/local/lib/python3.10/dist-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


# Start Telegram RVC

In [ ]:
# @title Download model
# @markdown Hugging Face or Google Drive
model_link = ""  # @param {type:"string"}

!python core.py download --model_link "{model_link}"

In [ ]:
# @title Enhanced Applio Telegram Bot with Full Output Display
# @markdown Shows all RVC processing output and errors directly in Telegram

import os
import asyncio
import logging
import subprocess
import json
from pathlib import Path
import threading
import time
from IPython.display import Audio, display, clear_output
import sys

# Install required packages
print("Installing Telegram bot dependencies...")
!pip install python-telegram-bot --upgrade -q
clear_output()

import telegram
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes, CallbackQueryHandler

# Configuration - UPDATE THESE VALUES
BOT_TOKEN = ""  # @param {type:"string"}
ADMIN_CHAT_ID = ""  # @param {type:"string"}

# Validate configuration
if BOT_TOKEN == "YOUR_BOT_TOKEN_HERE" or not BOT_TOKEN:
    print("❌ Please set your BOT_TOKEN!")
    raise SystemExit("Bot token not configured")

if ADMIN_CHAT_ID == "YOUR_CHAT_ID_HERE" or not ADMIN_CHAT_ID:
    print("❌ Please set your ADMIN_CHAT_ID!")
    raise SystemExit("Admin chat ID not configured")

# Paths
current_dir = "/content/Applio"
logs_dir = os.path.join(current_dir, "logs")

# Global variables
user_sessions = {}
bot_instance = None

# Setup logging
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

def log_print(message):
    """Print and log message"""
    print(message)
    logger.info(message)

def get_available_models():
    """Get list of available voice models"""
    log_print(f"🔍 Checking models in: {logs_dir}")

    if not os.path.exists(logs_dir):
        log_print(f"❌ Logs directory not found: {logs_dir}")
        return []

    models = []
    try:
        for item in os.listdir(logs_dir):
            model_path = os.path.join(logs_dir, item)
            if os.path.isdir(model_path):
                # Check for .pth file (required)
                files = os.listdir(model_path)
                has_pth = any(f.endswith('.pth') for f in files)
                if has_pth:
                    models.append(item)
                    log_print(f"✓ Found model: {item}")
    except Exception as e:
        log_print(f"❌ Error reading models: {e}")

    log_print(f"📊 Total models found: {len(models)}")
    return models

def get_model_files(model_name):
    """Get .pth and .index files for a model - FIXED VERSION"""
    model_folder = os.path.join(logs_dir, model_name)

    if not os.path.exists(model_folder):
        log_print(f"❌ Model folder not found: {model_folder}")
        return None, None

    try:
        files = os.listdir(model_folder)
        log_print(f"📁 Files in {model_name}: {files}")

        # Find .pth file
        pth_files = [f for f in files if f.endswith('.pth')]
        if pth_files:
            pth_path = os.path.join(model_folder, pth_files[0])
            log_print(f"✓ Found .pth file: {pth_files[0]}")
        else:
            log_print(f"❌ No .pth file found in {model_name}")
            return None, None

        # Find .index file (optional)
        index_files = [f for f in files if f.endswith('.index')]
        if index_files:
            index_path = os.path.join(model_folder, index_files[0])
            log_print(f"✓ Found .index file: {index_files[0]}")
        else:
            log_print(f"⚠️ No .index file found in {model_name} (optional)")
            index_path = None

        # Verify paths exist
        if pth_path and os.path.exists(pth_path):
            log_print(f"✅ .pth path verified: {pth_path}")
        else:
            log_print(f"❌ .pth path invalid: {pth_path}")
            return None, None

        if index_path and os.path.exists(index_path):
            log_print(f"✅ .index path verified: {index_path}")
        elif index_path:
            log_print(f"❌ .index path invalid: {index_path}")
            index_path = None

        return pth_path, index_path

    except Exception as e:
        log_print(f"❌ Error getting model files for {model_name}: {e}")
        return None, None

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Start command"""
    user_id = update.effective_user.id
    log_print(f"👤 User {user_id} started bot")

    # Initialize user session
    user_sessions[user_id] = {
        'selected_model': None,
        'processing': False,
        'pitch': 0,
        'volume_mix': 0.8,
        'index_rate': 0.7,
        'protect': 0.5,
        'show_debug': True  # New: Enable debug output by default
    }

    welcome_text = """
🎵 **Welcome to Enhanced Applio Voice Converter!**

**NEW FEATURE:** Full processing output visible! 📺

**How to use:**
1. `/models` - Select a voice model
2. Send any audio file to convert
3. Watch the live processing output! ✨

**Commands:**
• `/models` - Choose voice model
• `/settings` - Adjust parameters
• `/status` - Check current setup
• `/debug` - Toggle debug output
• `/help` - Show help

**Debug Mode:** ON (you'll see all RVC processing details)

Ready? Use `/models` to get started!
    """

    await update.message.reply_text(welcome_text, parse_mode='Markdown')

async def debug_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Toggle debug output"""
    user_id = update.effective_user.id

    if user_id not in user_sessions:
        await update.message.reply_text("❌ Use `/start` first!")
        return

    session = user_sessions[user_id]
    session['show_debug'] = not session.get('show_debug', True)

    status = "ON 📺" if session['show_debug'] else "OFF 🔇"
    await update.message.reply_text(
        f"🐛 **Debug Output:** {status}\n\n" +
        ("You'll see all RVC processing details during conversion." if session['show_debug']
         else "Processing output will be hidden."),
        parse_mode='Markdown'
    )

async def models_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Show available models"""
    user_id = update.effective_user.id

    models = get_available_models()

    if not models:
        await update.message.reply_text(
            "❌ **No models found!**\n\nMake sure you have downloaded models to `/content/Applio/logs/`"
        )
        return

    # Create buttons for models
    keyboard = []
    for model in models:
        keyboard.append([InlineKeyboardButton(f"🎭 {model}", callback_data=f"select_{model}")])

    keyboard.append([InlineKeyboardButton("🔄 Refresh", callback_data="refresh")])

    reply_markup = InlineKeyboardMarkup(keyboard)

    await update.message.reply_text(
        f"**🎭 Available Models ({len(models)}):**\n\nChoose a model for voice conversion:",
        reply_markup=reply_markup,
        parse_mode='Markdown'
    )

async def settings_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Show settings"""
    user_id = update.effective_user.id

    if user_id not in user_sessions:
        await update.message.reply_text("❌ Use `/start` first!")
        return

    session = user_sessions[user_id]

    settings_text = f"""
**⚙️ Current Settings:**

🎵 **Pitch:** `{session['pitch']}`
🔊 **Volume Mix:** `{session['volume_mix']}`
📊 **Index Rate:** `{session['index_rate']}`
🛡️ **Protect:** `{session['protect']}`
🐛 **Debug Output:** `{'ON' if session.get('show_debug', True) else 'OFF'}`

**Quick Pitch Settings:**
    """

    keyboard = [
        [InlineKeyboardButton("🔽 -12", callback_data="pitch_-12"),
         InlineKeyboardButton("🔽 -6", callback_data="pitch_-6"),
         InlineKeyboardButton("⚪ 0", callback_data="pitch_0")],
        [InlineKeyboardButton("🔼 +6", callback_data="pitch_6"),
         InlineKeyboardButton("🔼 +12", callback_data="pitch_12"),
         InlineKeyboardButton("🔄 Reset", callback_data="reset")]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)

    await update.message.reply_text(settings_text, reply_markup=reply_markup, parse_mode='Markdown')

async def status_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Show current status"""
    user_id = update.effective_user.id

    if user_id not in user_sessions:
        await update.message.reply_text("❌ Use `/start` first!")
        return

    session = user_sessions[user_id]
    models = get_available_models()

    status_text = f"""
**📊 Current Status:**

🎭 **Selected Model:** `{session['selected_model'] or 'None'}`
⚡ **Processing:** `{'Yes' if session['processing'] else 'No'}`
🤖 **Available Models:** `{len(models)}`
🐛 **Debug Output:** `{'ON' if session.get('show_debug', True) else 'OFF'}`

**Settings:**
🎵 Pitch: `{session['pitch']}`
🔊 Volume: `{session['volume_mix']}`
📊 Index: `{session['index_rate']}`
🛡️ Protect: `{session['protect']}`
    """

    await update.message.reply_text(status_text, parse_mode='Markdown')

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Help command"""
    help_text = """
**🎵 Enhanced Applio Voice Converter Help**

**Quick Start:**
1. `/models` - Select voice model
2. Send audio file
3. Watch live processing output! 📺

**Commands:**
• `/start` - Initialize bot
• `/models` - Choose model
• `/settings` - Adjust parameters
• `/status` - Check setup
• `/debug` - Toggle debug output
• `/help` - This help

**New Features:**
📺 **Live Output** - See all RVC processing steps
🐛 **Debug Mode** - Toggle detailed output
⚡ **Real-time** - Updates during processing

**Supported Audio:**
📥 **Input:** MP3, WAV, OGG, Voice messages
📤 **Output:** WAV format

**Tips:**
• Debug mode shows all processing details
• Use `+12` pitch for female voice
• Use `-12` pitch for male voice
• Watch for error messages in real-time

**Need help?** Contact admin!
    """

    await update.message.reply_text(help_text, parse_mode='Markdown')

async def button_callback(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle button clicks"""
    query = update.callback_query
    user_id = query.from_user.id
    data = query.data

    await query.answer()

    if user_id not in user_sessions:
        user_sessions[user_id] = {
            'selected_model': None,
            'processing': False,
            'pitch': 0,
            'volume_mix': 0.8,
            'index_rate': 0.7,
            'protect': 0.5,
            'show_debug': True
        }

    session = user_sessions[user_id]

    if data.startswith("select_"):
        # Model selection
        model_name = data.replace("select_", "")
        session['selected_model'] = model_name
        log_print(f"✅ User {user_id} selected model: {model_name}")

        # Test model files immediately
        pth_path, index_path = get_model_files(model_name)
        if pth_path:
            await query.edit_message_text(
                f"✅ **Model Selected:** `{model_name}`\n🎵 Now send any audio file to convert!\n📺 Debug output: {'ON' if session.get('show_debug') else 'OFF'}",
                parse_mode='Markdown'
            )
        else:
            await query.edit_message_text(
                f"❌ **Model Error:** `{model_name}`\n\nModel files not found or invalid.",
                parse_mode='Markdown'
            )

    elif data == "refresh":
        # Refresh models list
        models = get_available_models()
        if not models:
            await query.edit_message_text("❌ No models found!")
            return

        keyboard = []
        for model in models:
            keyboard.append([InlineKeyboardButton(f"🎭 {model}", callback_data=f"select_{model}")])
        keyboard.append([InlineKeyboardButton("🔄 Refresh", callback_data="refresh")])

        reply_markup = InlineKeyboardMarkup(keyboard)

        await query.edit_message_text(
            f"**🎭 Available Models ({len(models)}):**\n\nChoose a model:",
            reply_markup=reply_markup,
            parse_mode='Markdown'
        )

    elif data.startswith("pitch_"):
        # Pitch adjustment
        pitch_value = int(data.replace("pitch_", ""))
        session['pitch'] = pitch_value
        await query.edit_message_text(f"✅ Pitch set to: **{pitch_value}**", parse_mode='Markdown')

    elif data == "reset":
        # Reset settings
        session.update({
            'pitch': 0,
            'volume_mix': 0.8,
            'index_rate': 0.7,
            'protect': 0.5
        })
        await query.edit_message_text("✅ **Settings reset to defaults!**", parse_mode='Markdown')

async def send_telegram_message(context, chat_id, message, max_length=4000):
    """Send message to Telegram with length limit handling"""
    try:
        if len(message) <= max_length:
            await context.bot.send_message(chat_id=chat_id, text=message, parse_mode='Markdown')
        else:
            # Split long messages
            parts = [message[i:i+max_length] for i in range(0, len(message), max_length)]
            for i, part in enumerate(parts):
                if i == 0:
                    await context.bot.send_message(chat_id=chat_id, text=f"**Output (Part {i+1}/{len(parts)}):**\n```\n{part}\n```", parse_mode='Markdown')
                else:
                    await context.bot.send_message(chat_id=chat_id, text=f"```\n{part}\n```", parse_mode='Markdown')
    except Exception as e:
        # Fallback without markdown if formatting fails
        try:
            await context.bot.send_message(chat_id=chat_id, text=message[:max_length])
        except:
            await context.bot.send_message(chat_id=chat_id, text=f"Error sending output: {str(e)[:500]}")

async def handle_audio(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle audio files with full output display"""
    user_id = update.effective_user.id
    chat_id = update.effective_chat.id
    log_print(f"🎵 User {user_id} sent audio")

    # Check user session
    if user_id not in user_sessions:
        await update.message.reply_text("❌ Use `/start` first!")
        return

    session = user_sessions[user_id]

    # Check if model is selected
    if not session.get('selected_model'):
        await update.message.reply_text("❌ Select a model first with `/models`")
        return

    # Check if already processing
    if session.get('processing'):
        await update.message.reply_text("⏳ Already processing another file. Please wait...")
        return

    # Set processing flag
    session['processing'] = True

    try:
        # Send initial processing message
        if session.get('show_debug', True):
            status_msg = await update.message.reply_text(
                "🔄 **Starting Audio Processing...**\n\n📺 **Debug Mode ON** - You'll see all processing details!\n\n⏰ This may take 2-3 minutes",
                parse_mode='Markdown'
            )
        else:
            status_msg = await update.message.reply_text(
                "🔄 **Processing audio...**\n\nThis may take 2-3 minutes ⏰",
                parse_mode='Markdown'
            )

        # Get file
        if update.message.audio:
            file = update.message.audio
            extension = ".mp3"
        elif update.message.voice:
            file = update.message.voice
            extension = ".ogg"
        elif update.message.document and update.message.document.mime_type and 'audio' in update.message.document.mime_type:
            file = update.message.document
            extension = os.path.splitext(file.file_name or "audio.wav")[1] or ".wav"
        else:
            await status_msg.edit_text("❌ **Unsupported file type!**")
            session['processing'] = False
            return

        if session.get('show_debug'):
            await send_telegram_message(context, chat_id, f"📁 **File Details:**\n• Type: {extension}\n• Size: {getattr(file, 'file_size', 'Unknown')} bytes\n• Processing...")

        log_print(f"📁 Processing {extension} file")

        # Download file
        file_obj = await context.bot.get_file(file.file_id)
        input_path = f"/content/input_{user_id}{extension}"
        await file_obj.download_to_drive(input_path)

        if session.get('show_debug'):
            await send_telegram_message(context, chat_id, f"⬇️ **Download Complete:**\n• Path: `{input_path}`\n• Size: {os.path.getsize(input_path)} bytes")

        log_print(f"⬇️ Downloaded to: {input_path}")

        # Convert to WAV if needed
        if not input_path.endswith('.wav'):
            wav_path = f"/content/input_{user_id}.wav"

            if session.get('show_debug'):
                await send_telegram_message(context, chat_id, f"🔄 **Converting to WAV:**\n• From: {extension}\n• To: .wav\n• Sample rate: 44100 Hz")

            try:
                result = subprocess.run([
                    'ffmpeg', '-i', input_path, '-ar', '44100', '-ac', '1', '-y', wav_path
                ], check=True, capture_output=True, text=True)

                if session.get('show_debug') and result.stderr:
                    await send_telegram_message(context, chat_id, f"🔧 **FFmpeg Output:**\n```\n{result.stderr[-1000:]}\n```")

                if os.path.exists(wav_path):
                    os.remove(input_path)
                    input_path = wav_path

                    if session.get('show_debug'):
                        await send_telegram_message(context, chat_id, f"✅ **Conversion Success:**\n• New path: `{input_path}`\n• New size: {os.path.getsize(input_path)} bytes")

                    log_print(f"🔄 Converted to WAV: {input_path}")
            except Exception as e:
                error_msg = f"⚠️ FFmpeg conversion failed: {e}"
                log_print(error_msg)
                if session.get('show_debug'):
                    await send_telegram_message(context, chat_id, f"⚠️ **Conversion Warning:**\n{error_msg}\n\nContinuing with original file...")

        # Process with Applio
        success = await process_with_applio_enhanced(user_id, input_path, session, status_msg, context, chat_id)

        if success:
            output_path = f"/content/output_{user_id}.wav"

            # Send result info
            if session.get('show_debug'):
                output_size = os.path.getsize(output_path) if os.path.exists(output_path) else 0
                await send_telegram_message(context, chat_id, f"📤 **Conversion Complete!**\n• Output size: {output_size} bytes\n• Model: `{session['selected_model']}`\n• Pitch: `{session['pitch']}`\n\nSending audio file...")
            else:
                await status_msg.edit_text("📤 **Sending converted audio...**", parse_mode='Markdown')

            with open(output_path, 'rb') as audio_file:
                await context.bot.send_audio(
                    chat_id=chat_id,
                    audio=audio_file,
                    caption=f"🎵 **Converted with:** `{session['selected_model']}`\n🎛️ **Pitch:** `{session['pitch']}`\n📺 **Debug mode:** `{'ON' if session.get('show_debug') else 'OFF'}`",
                    parse_mode='Markdown'
                )

            log_print(f"✅ Successfully sent result to user {user_id}")

            # Cleanup
            for path in [input_path, output_path]:
                if os.path.exists(path):
                    os.remove(path)

            if session.get('show_debug'):
                await send_telegram_message(context, chat_id, "🧹 **Cleanup Complete** - Temporary files removed")

            await status_msg.delete()
        else:
            await status_msg.edit_text(
                f"❌ **Conversion failed!**\n\nTry:\n• Different model\n• Shorter audio file\n• Check if model `{session['selected_model']}` exists\n\n📺 Check debug output above for details."
            )

    except Exception as e:
        error_msg = f"❌ Audio processing error: {e}"
        log_print(error_msg)

        if session.get('show_debug'):
            await send_telegram_message(context, chat_id, f"**💥 Processing Error:**\n```\n{str(e)}\n```")
        else:
            await update.message.reply_text(f"❌ **Error:** {str(e)}")

    finally:
        session['processing'] = False

async def process_with_applio_enhanced(user_id, input_path, session, status_msg, context, chat_id):
    """Process audio using Applio core.py with full output display"""
    try:
        show_debug = session.get('show_debug', True)
        model_name = session['selected_model']

        if show_debug:
            await send_telegram_message(context, chat_id, f"🎭 **Starting RVC Processing:**\n• Model: `{model_name}`\n• Checking model files...")

        log_print(f"🎭 Processing with model: {model_name}")

        # Get model files with proper error handling
        pth_path, index_path = get_model_files(model_name)

        if not pth_path:
            error_msg = f"❌ Model files not found for: {model_name}"
            log_print(error_msg)
            if show_debug:
                await send_telegram_message(context, chat_id, f"**❌ Model Error:**\n{error_msg}")
            return False

        if show_debug:
            await send_telegram_message(context, chat_id, f"✅ **Model Files Found:**\n• PTH: `{os.path.basename(pth_path)}`\n• Index: `{os.path.basename(index_path) if index_path else 'None'}`")

        # Verify pth_path is a string, not tuple
        if not isinstance(pth_path, str):
            error_msg = f"❌ Invalid pth_path type: {type(pth_path)} - {pth_path}"
            log_print(error_msg)
            if show_debug:
                await send_telegram_message(context, chat_id, f"**❌ Path Error:**\n{error_msg}")
            return False

        if not os.path.exists(pth_path):
            error_msg = f"❌ .pth file does not exist: {pth_path}"
            log_print(error_msg)
            if show_debug:
                await send_telegram_message(context, chat_id, f"**❌ File Missing:**\n{error_msg}")
            return False

        log_print(f"📁 Using .pth: {pth_path}")

        # Handle index path
        if index_path:
            if not isinstance(index_path, str):
                log_print(f"⚠️ Invalid index_path type, ignoring: {type(index_path)}")
                index_path = None
            elif not os.path.exists(index_path):
                log_print(f"⚠️ .index file does not exist, ignoring: {index_path}")
                index_path = None
            else:
                log_print(f"📊 Using .index: {index_path}")

        output_path = f"/content/output_{user_id}.wav"

        # Show processing parameters
        if show_debug:
            params_text = f"""🎛️ **Processing Parameters:**
• Pitch: `{session['pitch']}`
• Volume Mix: `{session['volume_mix']}`
• Index Rate: `{session['index_rate']}`
• Protect: `{session['protect']}`
• F0 Method: `rmvpe`
• Output Format: `WAV`"""
            await send_telegram_message(context, chat_id, params_text)

        # Build Applio command
        cmd = [
            "python", "core.py", "infer",
            "--pitch", str(session['pitch']),
            "--volume_envelope", str(session['volume_mix']),
            "--index_rate", str(session['index_rate']),
            "--hop_length", "128",
            "--protect", str(session['protect']),
            "--f0_autotune", "False",
            "--f0_method", "rmvpe",
            "--input_path", input_path,
            "--output_path", output_path,
            "--pth_path", pth_path,
            "--split_audio", "False",
            "--clean_audio", "False",
            "--clean_strength", "0.7",
            "--export_format", "WAV",
            "--embedder_model", "contentvec",
            "--embedder_model_custom", "",
            "--formant_shifting", "False",
            "--formant_qfrency", "1.0",
            "--formant_timbre", "1.0",
            "--post_process", "False",
            "--reverb", "False",
            "--pitch_shift", "False",
            "--limiter", "False",
            "--gain", "False",
            "--distortion", "False",
            "--chorus", "False",
            "--bitcrush", "False",
            "--clipping", "False",
            "--compressor", "False",
            "--delay", "False",
            "--reverb_room_size", "0.5",
            "--reverb_damping", "0.5",
            "--reverb_wet_gain", "0.0",
            "--reverb_dry_gain", "0.0",
            "--reverb_width", "1.0",
            "--reverb_freeze_mode", "0.0",
            "--pitch_shift_semitones", "0.0",
            "--limiter_threshold", "-1.0",
            "--limiter_release_time", "0.05",
            "--gain_db", "0.0",
            "--distortion_gain", "0.0",
            "--chorus_rate", "1.5",
            "--chorus_depth", "0.1",
            "--chorus_center_delay", "15.0",
            "--chorus_feedback", "0.25",
            "--chorus_mix", "0.5",
            "--bitcrush_bit_depth", "4",
            "--clipping_threshold", "0.5",
            "--compressor_threshold", "-20.0",
            "--compressor_ratio", "4.0",
            "--compressor_attack", "0.001",
            "--compressor_release", "0.1",
            "--delay_seconds", "0.1",
            "--delay_feedback", "0.5",
            "--delay_mix", "0.5"
        ]

        # Add index path only if it exists and is valid
        if index_path and isinstance(index_path, str) and os.path.exists(index_path):
            cmd.extend(["--index_path", index_path])
            log_print("✅ Added index path to command")
            if show_debug:
                await send_telegram_message(context, chat_id, "✅ **Index file** will be used for better quality")
        else:
            log_print("⚠️ No valid index path, proceeding without it")
            if show_debug:
                await send_telegram_message(context, chat_id, "⚠️ **No index file** - proceeding without it (may affect quality)")

        if show_debug:
            await send_telegram_message(context, chat_id, f"🚀 **Starting RVC Core Process:**\n• Command: `python core.py infer`\n• Parameters: {len(cmd)-3} arguments\n• Timeout: 5 minutes")

        log_print(f"🔄 Running Applio command...")
        log_print(f"📝 Command: python core.py infer [with {len(cmd)-3} parameters]")

        # Update status
        await status_msg.edit_text("🤖 **Running AI conversion...**\n\n📺 Watch for live output below!", parse_mode='Markdown')

        # Run the command with real-time output capture
        process = subprocess.Popen(
            cmd,
            cwd=current_dir,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,  # Combine stderr with stdout
            text=True,
            universal_newlines=True,
            bufsize=1  # Line buffered
        )

        output_lines = []
        last_telegram_update = time.time()

        # Read output line by line in real-time
        while True:
            line = process.stdout.readline()
            if not line and process.poll() is not None:
                break

            if line.strip():
                output_lines.append(line.strip())
                log_print(f"📤 RVC: {line.strip()}")

                # Send updates to Telegram every 3 seconds or when important info appears
                current_time = time.time()
                is_important = any(keyword in line.lower() for keyword in [
                    'error', 'fail', 'exception', 'loading', 'processing', 'complete',
                    'model', 'audio', 'inference', 'pitch', 'index', 'converting'
                ])

                if show_debug and (current_time - last_telegram_update > 3 or is_important):
                    # Send recent output (last 5 lines)
                    recent_output = '\n'.join(output_lines[-5:])
                    if recent_output:
                        await send_telegram_message(context, chat_id, f"🔄 **RVC Live Output:**\n```\n{recent_output[-1500:]}\n```")
                        last_telegram_update = current_time

        # Wait for process to complete
        return_code = process.wait()

        # Get any remaining output
        remaining_output, _ = process.communicate()
        if remaining_output:
            output_lines.extend(remaining_output.strip().split('\n'))

        log_print(f"📊 Command completed with return code: {return_code}")

        # Send final output summary
        if show_debug:
            final_output = '\n'.join(output_lines[-10:]) if output_lines else "No output captured"
            await send_telegram_message(context, chat_id, f"🏁 **RVC Process Complete:**\n• Return Code: `{return_code}`\n• Final Output:\n```\n{final_output[-1500:]}\n```")

        # Check if output file was created
        if os.path.exists(output_path):
            file_size = os.path.getsize(output_path)
            log_print(f"✅ Output file created: {output_path} ({file_size} bytes)")

            if show_debug:
                await send_telegram_message(context, chat_id, f"✅ **Success!**\n• Output file: `{output_path}`\n• Size: {file_size} bytes\n• Ready to send!")

            return True
        else:
            log_print(f"❌ Output file not created: {output_path}")

            if show_debug:
                error_summary = '\n'.join([line for line in output_lines if any(keyword in line.lower() for keyword in ['error', 'fail', 'exception'])])
                if error_summary:
                    await send_telegram_message(context, chat_id, f"❌ **Processing Failed:**\n• No output file created\n• Errors found:\n```\n{error_summary[-1000:]}\n```")
                else:
                    await send_telegram_message(context, chat_id, f"❌ **Processing Failed:**\n• No output file created\n• No specific errors found\n• Check full output above")

            return False

    except subprocess.TimeoutExpired:
        error_msg = "⏰ Command timed out after 5 minutes"
        log_print(error_msg)
        if show_debug:
            await send_telegram_message(context, chat_id, f"**⏰ Timeout Error:**\n{error_msg}\n\nTry with a shorter audio file.")
        return False
    except Exception as e:
        error_msg = f"❌ Processing error: {e}"
        log_print(error_msg)
        if show_debug:
            await send_telegram_message(context, chat_id, f"**💥 Exception Error:**\n```\n{str(e)}\n```")
        return False

# Bot runner
async def run_bot():
    """Run the Telegram bot"""
    try:
        # Create application
        app = Application.builder().token(BOT_TOKEN).build()

        # Add handlers
        app.add_handler(CommandHandler("start", start_command))
        app.add_handler(CommandHandler("models", models_command))
        app.add_handler(CommandHandler("settings", settings_command))
        app.add_handler(CommandHandler("status", status_command))
        app.add_handler(CommandHandler("debug", debug_command))  # New debug command
        app.add_handler(CommandHandler("help", help_command))
        app.add_handler(CallbackQueryHandler(button_callback))
        app.add_handler(MessageHandler(
            filters.AUDIO | filters.VOICE | (filters.Document.AUDIO),
            handle_audio
        ))

        models = get_available_models()
        print("🤖 Enhanced Applio Telegram Bot Starting...")
        print(f"📊 Found {len(models)} models: {models}")
        print(f"📁 Applio directory: {current_dir}")
        print("📺 NEW: Full RVC output display enabled!")
        print("✅ Bot is ready! Send /start to begin.")

        # Start bot
        await app.initialize()
        await app.start()
        await app.updater.start_polling()

        # Keep running
        while True:
            await asyncio.sleep(1)

    except Exception as e:
        print(f"❌ Bot error: {e}")
        logger.error(f"Bot error: {e}")

# Start the bot
print("🚀 Starting Enhanced Applio Telegram Bot...")
print("📺 NEW FEATURES:")
print("   • Live RVC processing output")
print("   • Real-time error detection")
print("   • Debug mode toggle")
print("   • Full command output display")
print("🔧 Make sure your BOT_TOKEN and ADMIN_CHAT_ID are correct above")

# Run bot in Colab environment
try:
    # Check for existing event loop (Colab compatibility)
    try:
        loop = asyncio.get_running_loop()
        print("📋 Using existing event loop")
        task = asyncio.create_task(run_bot())
        print("✅ Enhanced bot started as background task!")
        print("🔄 Bot will run until you stop the cell or restart runtime")
        print("📺 Users will now see all RVC processing details!")
    except RuntimeError:
        print("📋 Creating new event loop")
        asyncio.run(run_bot())

except KeyboardInterrupt:
    print("🛑 Bot stopped by user")
except Exception as e:
    print(f"❌ Startup error: {e}")
    print("💡 Try restarting the cell if issues persist")